CELL 1 — Load Silver Fact Table

In [ ]:
storage = "your-storage-name"

silver_order_details = f"abfss://datalake@{storage}.dfs.core.windows.net/silver/fakestore/order_details/"

df_fact = spark.read.parquet(silver_order_details)
df_fact.createOrReplaceTempView("order_details")


StatementMeta(spkecommerce, 9, 8, Finished, Available, Finished)

CELL 2 — GOLD TABLE: DAILY REVENUE

In [8]:
df_daily_revenue = spark.sql("""
    SELECT 
        date(order_timestamp) AS order_date,
        SUM(total_amount) AS total_revenue,
        COUNT(*) AS total_items_sold
    FROM order_details
    GROUP BY date(order_timestamp)
    ORDER BY order_date
""")

df_daily_revenue.write.mode("overwrite").parquet(
    f"abfss://datalake@{storage}.dfs.core.windows.net/gold/fakestore/daily_revenue/"
)

display(df_daily_revenue)
print("Gold table: daily_revenue created")


StatementMeta(spkecommerce, 9, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a49c0345-bf51-474f-bba6-c7a8d9074339)

Gold table: daily_revenue created


CELL 3 — GOLD TABLE: TOP SELLING PRODUCTS

In [9]:
df_product_sales = spark.sql("""
    SELECT 
        product_id,
        product_title,
        SUM(quantity) AS total_units_sold,
        SUM(total_amount) AS total_sales
    FROM order_details
    GROUP BY product_id, product_title
    ORDER BY total_units_sold DESC
""")

df_product_sales.write.mode("overwrite").parquet(
    f"abfss://datalake@{storage}.dfs.core.windows.net/gold/fakestore/product_sales/"
)

display(df_product_sales.limit(20))
print("Gold table: product_sales created")


StatementMeta(spkecommerce, 9, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d879ab28-3934-4043-bcb8-a180116d981b)

Gold table: product_sales created


CELL 5 — GOLD TABLE: CATEGORY SALES

In [10]:
df_category_sales = spark.sql("""
    SELECT
        product_category,
        SUM(quantity) AS total_units_sold,
        SUM(total_amount) AS total_revenue
    FROM order_details
    GROUP BY product_category
    ORDER BY total_revenue DESC
""")

df_category_sales.write.mode("overwrite").parquet(
    f"abfss://datalake@{storage}.dfs.core.windows.net/gold/fakestore/category_sales/"
)

display(df_category_sales)
print("Gold table: category_sales created")


StatementMeta(spkecommerce, 9, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b0136b8a-5611-438d-9de1-f3f57719534f)

Gold table: category_sales created
